In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, time
import pytz

# ---------------- LOAD DATA ----------------
df = pd.read_csv("Play Store Data.csv")

def show_installs_time_series(df, input_time_ist):
    # Parse user input time
    user_time = datetime.strptime(input_time_ist, "%H:%M").time()
    start_time = time(18, 0)   # 6 PM 18
    end_time   = time(21, 0)   # 9 PM 21

    # Restrict chart visibility
    if not (start_time <= user_time <= end_time):
        print("⏰ Chart visible only between 6 PM and 9 PM IST.")
        return

    df = df.copy()

    # Clean installs
    def clean_installs(val):
        try:
            return int(float(str(val).replace(",", "").replace("+", "")))
        except:
            return None

    df["Installs_Clean"] = df["Installs"].apply(clean_installs)
    df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")
    df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")

    # Drop invalid rows
    df = df.dropna(subset=["Installs_Clean", "Last Updated", "Reviews"])

    # Apply filters
    df = df[
        (df["Reviews"] > 500) &
        (~df["App"].astype(str).str.upper().str.startswith(("X", "Y", "Z"))) &
        (~df["App"].astype(str).str.contains("S", case=False)) &
        (df["Category"].astype(str).str.upper().str.startswith(("E", "C", "B")))
    ]

    if df.empty:
        print("⚠️ No data matches the filters.")
        return

    # Translate categories
    category_translation = {
        "BEAUTY": "सौंदर्य",       # Hindi
        "BUSINESS": "வணிகம்",     # Tamil
        "DATING": "Partnersuche"  # German
    }
    df["Category_Display"] = df["Category"].str.upper().replace(category_translation)

    # Monthly installs
    df["Month"] = df["Last Updated"].dt.to_period("M").dt.to_timestamp()
    grouped = (
        df.groupby(["Month", "Category_Display"])["Installs_Clean"]
        .sum()
        .reset_index()
        .sort_values("Month")
    )

    # Calculate growth
    pivot = grouped.pivot(index="Month", columns="Category_Display", values="Installs_Clean").fillna(0)
    growth = pivot.pct_change()
    highlight_months = growth.gt(0.20).any(axis=1)

    # Plot line chart
    fig = px.line(
        grouped,
        x="Month",
        y="Installs_Clean",
        color="Category_Display",
        title="Total Installs Trend Over Time by Category"
    )

    # Highlight growth >20%
    for month, flag in zip(pivot.index, highlight_months):
        if flag:
            fig.add_vrect(
                x0=month,
                x1=month + pd.offsets.MonthEnd(1),
                fillcolor="yellow",
                opacity=0.25,
                layer="below",
                line_width=0
            )

    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Total Installs",
        template="plotly_white"
    )

    fig.show()

# ✅ CALL FUNCTION
show_installs_time_series(df, "18:30")


⏰ Chart visible only between 6 PM and 9 PM IST.
